<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/PostProcessing_AnalisiPicchiCO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!mkdir -p local_modules/postprocessing_CO2

In [3]:
%%writefile local_modules/postprocessing_CO2/__init__.py

import arff
import csv
import numpy as np

def get_postprocessing_CO2(pred, val):

  #Carico solamente i valori della CO2 e del datetime dell'insieme di test

  #LOAD SENSOR DATA
  datasetSensor = '/root/data/uHooComplete_featureDataset_Reduced.arff'

  with open (datasetSensor, encoding='utf-8') as fs:
    dataSensor = arff.load(fs)

  dataS = np.array(dataSensor['data'])

  #CONVERTING VALUES
  for i in dataS:
    if(i[-1] == 'Other'): i[-1] = 0
    else : i[-1] = 1

  #TEST SENSOR DATA  
  dataT = dataS[val+1:]
  datastr = dataT[:,1]
  datastr2 = dataT[:,2]
  co2 = datastr.astype('float32')
  time = datastr2.astype('float32')

  #POST PROCESSING
  pasto = []
  index =[]
  p = []

  for i in pred:
    p.append(i)
    
  for i in range(0, len(pred)):
    if p[i] == 1 and i+1 in range(0, len(pred)) and time[i+1]>=time[i]:    
      index.append(i)
      pasto.append(co2[i])

      if (i+1) in range(0, len(pred)) and p[i+1] == 0: 
        if len(pasto) > 4:
          pasto = np.asarray(pasto)
          minimum = np.amin(pasto)
          maximum = np.amax(pasto)
          pasto = pasto.tolist()
          index_max = pasto.index(maximum)
          index_min = pasto.index(minimum)
          
          for j in range(index[0], index[index_min] + 1):
            p[j] = 0

          for j in range(index[index_max], index[-1]):
            if(j<index[-1]):
              p[j+1] = 0
        
        pasto=[]
        index=[]
  
  return p

Writing local_modules/postprocessing_CO2/__init__.py
